# Disegno di una porzione di un quartiere

Planimetria di riferimento
<img src="planimetria.jpg">
<img src="planimetriaFinale.png">

dettaglio finestra (con utilizzo della funzione materials per il vetro)
<img src="vetro.png">

dettaglio di una parte della planimetria con lampioni
<img src="lampione.png">

<img src="alberi1.png">

In [50]:
from pyplasm import *
from casa import *
from tetto import *

In [51]:
def Falbero(k):
    ramo = MKPOL([[[0,0,0],[0,0,1],[1,0,0]],[[1,2,3]],[[1]]])
    ramo2 = MKPOL([[[0,0,0],[0,0,1.5],[1.5,0,0]],[[1,2,3]],[[1]]])
    ramo3 = MKPOL([[[0,0,0],[0,0,2],[2,0,0]],[[1,2,3]],[[1]]])
    ramo = TEXTURE("albero.jpeg")(ramo)
    ramo2 = TEXTURE("albero.jpeg")(ramo2)
    ramo3 = TEXTURE("albero.jpeg")(ramo3)
    fronda = STRUCT([ramo3, T(3)(1.7), ramo2, T(3)(1.2), ramo])
    fronda = STRUCT([fronda, S(1)(-1)(fronda)])
    tot = fronda
    for r in range(1,k):
        tot = STRUCT([tot, R([1,2])(3.14*r/k), fronda])
    tot = S([1,2,3])([2,2,2.5])(tot)
    return tot

def FtroncoChioma(k):
    chioma = CIRCLE(3)([24,32])
    chioma = TEXTURE("albero.jpeg")(chioma)
    for r in range(1,k):
        chioma = STRUCT([chioma, R([1,3])(3.14*r/k), chioma])
    chioma = R([2,3])(3.14/2)(chioma)
    tronco = CYLINDER([.35, 7.5])(8)
    ambient = [0.0, 0.0, 0.0, 1.0]
    diffuse = [150./255, 70./255, 0.0, 1.0]
    specular = [0.0225, 0.0225, 0.0225, 1.0]
    emission = [0.0, 0.0, 0.0, 1.0]
    shininess = [12.8]
    tronco = MATERIAL(ambient+diffuse+specular+emission+shininess)(tronco)
    albero = STRUCT([tronco, T(3)(6.5), chioma])
    return albero
    
def Flampione():
    ambient = [0.05375, 0.05, 0.06625, 0.82]
    diffuse = [0., 1., 0., 1.]
    specular = [0.332741, 0.328634, 0.346435, 0.82]
    emission = [0.0,0.0,0.0,0.0]
    shininess = [38.4]
    palo = MATERIAL(ambient+diffuse+specular+emission+shininess)(CYLINDER([.05, 1.8])(10))
    #vetro
    ambient = [0,0,0,1]
    diffuse = [0.588235, 0.670588, 0.729412, .8]
    specular = [.7,.7,.7,1]
    #specular = [.9,.9,.9,1]
    #emission = [ 0,0,0,1]
    emission = [ .8,.8,.8,1]
    shininess = [96]
    paralume = MATERIAL(ambient+diffuse+specular+emission+shininess)(SPHERE(.35)([24,32]))
    ambient = [.5,.5,.5,1]
    diffuse = [240./255, 232./255, 64./255, 1.]
    luce = MATERIAL(ambient+diffuse+specular+emission+shininess)(SPHERE(.07)([24,32]))
    lampione = STRUCT([palo, T(3)(1.87), luce, T(3)(.09), paralume])
    return lampione

In [52]:
def Fcasa(k, np):
    dx = 1.
    dy = 2.5
    dz = 1.7
    n = dy/.25
    n -= n%1
    riserStep = dz/n
    ax = appartamento(2)[1]
    ay = appartamento(2)[2]
    p1 = [.0, .0, .0]
    p2 = [ax+2*ay, 0., 0.]
    p3 = [0., ax, .0]
    p4 = [ax+2*ay, ax, .0]
    p5 = [ax/4., 0, 2.]
    p6 = [0, ay/4., 2.]
    p7 = [2*ay+(3./4)*ax, 0., 2.]
    p8 = [0,(3./4)*ax, 2.]
    piano1 = trovaPiano(p1, p2, p6)
    piano2 = trovaPiano(p1, p3, p5)
    piano3 = trovaPiano(p2, p4, p7)
    piano4 = trovaPiano(p3, p4, p8)
    i1 = intersezioniPiani(piano1, piano3, 2.)
    i2 = intersezioniPiani(piano3, piano4, 2.)
    i3 = intersezioniPiani(piano4, piano2, 2.)
    i4 = intersezioniPiani(piano2, piano1, 2.)
    i = [i1, i2, i3, i4]
    pti = []
    tettoia = []
    for el in i:
        tettoia.append(el[1])
        for p in el:
            pti.append(p)

    pol = [[1,2,3,4],[3,4,5,6],[5,6,7,8],[7,8,1,2]]
    d = MKPOL([pti, pol, [[1]]])
    d = TEXTURE("roof.jpg")(d)

    rette = []
    for r in range (0, len(tettoia)):
        tmp = []
        if r == len(tettoia) - 1:
            tmp.append(retta(tettoia[r], tettoia[0]))
            tmp.append([tettoia[r][0], tettoia[r][1]])
            tmp.append([tettoia[0][0], tettoia[0][1]])
        else:
            tmp.append(retta(tettoia[r], tettoia[r+1]))
            tmp.append([tettoia[r][0], tettoia[r][1]])
            tmp.append([tettoia[r+1][0], tettoia[r+1][1]])
        rette.append(tmp)

    ordinate = ordinaRette(rette)
    t = disegnaPiano(ordinate)
    tetto = STRUCT([d, T(3)(2.), t])
    tetto = S([1,2,3])(.8)(tetto)
    casa = numeroPiani(k, dx, dy, dz, np)
    casa = STRUCT([casa, T([1,2,3])([-appartamento(2)[2],-appartamento(2)[1]/3.+.3,4*(dz-riserStep)]),tetto])
    return casa

In [53]:
def ggpl_w11():
    punti = []
    punti.append([[0,0],[0,100]])
    punti.append([[4, 80],[170,110],[160,0]])
    punti.append([[134.5,70], [145, 78], [145, 85]])
    punti.append([[151,10], [151,15]])

    parcheggi = []
    parcheggi.append([[140,85],[140,110]])

    parcheggiPiccoli = []
    parcheggiPiccoli.append([[131, 2],[131, 20]])
    parcheggiPiccoli.append([[18, 85],[18, 93]])

    stradineOrizzontali = []
    #case 1 2 3 4
    stradineOrizzontali.append([[6,30],[109.3,30]])
    #case 7 8 9 10
    stradineOrizzontali.append([[6,42.5],[102,42.5]])
    stradineOrizzontali.append([[102,30],[105.5,33.5],[107.5,37.5],[105.5,43.5],[102,45]])
    stradineOrizzontali.append([[101,44.5],[103,95]])
    stradineOrizzontali.append([[97.5,95],[110,95]])
    #casa 11
    stradineOrizzontali.append([[100,96],[100,115]])
    #casa 16
    stradineOrizzontali.append([[110,96],[110,115]])
    #case 12 13 14 15
    stradineOrizzontali.append([[50,115],[132,115]])
    stradineOrizzontali.append([[100.5,65],[110,59]])
    #case 5 6
    stradineOrizzontali.append([[109.3,61],[109.3,30]])
    stradineOrizzontali.append([[109.3,31],[118,25],[118,20]])
    stradineOrizzontali.append([[118,20],[118,12],[132,12]])

    stradeAppartamenti = []
    #case 12 13 14 15
    stradeAppartamenti.append([[50,115],[50, 135]])
    stradeAppartamenti.append([[77,115],[77, 135]])
    stradeAppartamenti.append([[104,115],[104, 135]])
    stradeAppartamenti.append([[132,115],[132, 135]])
    #case 7 8 9 10
    stradeAppartamenti.append([[27,45],[27, 58]])
    stradeAppartamenti.append([[46,45],[46, 55]])
    stradeAppartamenti.append([[65,45],[65, 55]])
    stradeAppartamenti.append([[84,45],[84, 55]])
    #case 1 2 3 4
    stradeAppartamenti.append([[27,30],[27, 20]])
    stradeAppartamenti.append([[46,30],[46, 20]])
    stradeAppartamenti.append([[65,30],[65, 20]])
    stradeAppartamenti.append([[84,30],[84, 20]])
    #casa 11
    stradeAppartamenti.append([[100,102],[88, 102]])
    #casa 16
    stradeAppartamenti.append([[110,100],[119, 100]])
    #case 5 6
    stradeAppartamenti.append([[109.3,50],[122, 50]])
    stradeAppartamenti.append([[109.3,35],[122, 35]])

    #vertices=[[0,0],[1.5,0],[-1,2],[2,2],[2,0]]
    #VIEW(Plasm.Struct([ Plasm.power(BEZIERSTRIPE([punti,0.25,50]),QUOTE([.1])), a]))
    bigBezier = []
    for el in punti:
        bigBezier.append(BEZIERSTRIPE([el,-6,30]))
    for parcheggio in parcheggi:
        bigBezier.append(BEZIERSTRIPE([parcheggio, -40, 30]))
    for parcheggio in parcheggiPiccoli:
        bigBezier.append(BEZIERSTRIPE([parcheggio, -20, 30]))
    for stradina in stradineOrizzontali:
        bigBezier.append(BEZIERSTRIPE([stradina, 2.5, 30]))
    for strada in stradeAppartamenti:
        bigBezier.append(BEZIERSTRIPE([strada, 1.5, 30]))
    tot = STRUCT(bigBezier)

    ambient = [.0, .0, .0, 1.0]
    diffuse = [65./255, 68./255, 67./255, 1.0]
    specular = [0.0425, 0.0425, 0.0425, 1.0]
    emission = [0,0,0,.3]
    shininess = [12.8]

    tot = MATERIAL(ambient+diffuse+specular+emission+shininess)(tot)
    casa = Fcasa(2,2)
    #1
    tot = STRUCT([tot,T([1,2])([23,19]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #2
    tot = STRUCT([tot,T([1,2])([42,19]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #3
    tot = STRUCT([tot,T([1,2])([70,13]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])
    #4
    tot = STRUCT([tot,T([1,2])([89,13]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])

    #5
    tot = STRUCT([tot,T([1,2])([120,40]), casa])
    #6
    tot = STRUCT([tot,T([1,2])([120,55]), casa])

    #10
    tot = STRUCT([tot,T([1,2])([22,63]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #9
    tot = STRUCT([tot,T([1,2])([40,60]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #8
    tot = STRUCT([tot,T([1,2])([70,54]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])
    #7
    tot = STRUCT([tot,T([1,2])([89,54]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])

    #11
    tot = STRUCT([tot,T([1,2])([83,106]), casa])

    #12
    tot = STRUCT([tot,T([1,2])([45,136]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #13
    tot = STRUCT([tot,T([1,2])([72,136]), R([1,2])(-math.pi/2)(S(2)(-1)(casa))])
    #14
    tot = STRUCT([tot,T([1,2])([107,131]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])
    #15
    tot = STRUCT([tot,T([1,2])([135,131]), R([1,2])(math.pi/2)(S(2)(-1)(casa))])
    #16
    tot = STRUCT([tot,T([1,2])([118,105]), casa])

    albero = Falbero(5)
    chioma = FtroncoChioma(4)
    lampione = Flampione()

    for r in range(0, 4):
        tot = STRUCT([tot, T([1,2])([6, 10 + r*30]),lampione])
        tot = STRUCT([tot, T([1,2])([51 + r*27, 118]),lampione])
    for r in range(1, 6):
        tot = STRUCT([tot, T([1,2])([r*20, 29.5]), lampione])
        if r == 5: tot = STRUCT([tot, T([1,2])([r*20 - 3, 45.5]), lampione])
        else: tot = STRUCT([tot, T([1,2])([r*20, 45.5]), lampione])

    tot = STRUCT([tot, T([1,2])([110, 29.5]), lampione])

    for r in range(0,5):
        tot = STRUCT([tot,T([1,2])([166, r*15]),chioma])

    for r in range(1,5):
        tot = STRUCT([tot, T([1,2])([98 + 0.6 * r, 45.5 + 16*r]), lampione])
    for r in range(1,10):
        tot = STRUCT([tot,T([1,2])([5 + 10*r, 37.5]), albero])

    box = UKPOL(BOX([1,2])(tot))[0]
    minX = 100
    maxX = 0
    minY = 100
    maxY = 0
    for el in box:
        if el[0] > maxX: maxX = el[0]
        if el[0] < minX: minX = el[0]
        if el[1] > maxY: maxY = el[1]
        if el[1] < minY: minY = el[1]
    LX = maxX - minX
    LY = maxY - minY
    cubo = TEXTURE("grass.jpg")(CUBOID([LX, LY, .1]))
    tot = STRUCT([tot, T([1,2,3])([minX, minY, -.2]), cubo])
    return tot
#VIEW(tot)

In [54]:
VIEW(ggpl_w11())

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000B1A97B0> >